In [46]:
# import delle librerie
import cv2
import numpy as np

In [47]:
def assign_sets():
    '''
    Questa funzione legge dalla cartella i seguenti file:
    - test_assignment.txt
    - train_assignment.txt
    - val_assignment.txt
    Poi ritorna tre array di valori
    '''
    test_set = []
    train_set = []
    val_set = []
    # leggo prima Train
    with open('train_assignment.txt', 'r') as f:
        for row in f:
            row = row.strip()
            train_set.append(row)
    # poi test
    with open('test_assignment.txt', 'r') as f:
        for row in f:
            row = row.strip()
            test_set.append(row)
    # infine validation
    with open('val_assignment.txt', 'r') as f:
        for row in f:
            row = row.strip()
            val_set.append(row)
    
    print(f'Train set: {len(train_set)}')
    print(f'Test set: {len(test_set)}')
    print(f'Val set: {len(val_set)}')
    return train_set, test_set, val_set

In [48]:
def extract_boundary_box(valore, annotations):
    '''
    Questa funzione prende un valore in input, ovvero l'immagine dove cercare
    ed estrae le boundary box della classe 1 relative a quel valore.
    '''
    path_file = 'Annotations/' + valore + '.jpg.txt'
    # print(path_file)
    rows = []
    if path_file[12:] in annotations:
        # il file è presente in Annotations
        with open(path_file, 'r') as f:
            for row in f:
                row = row.strip()
                rows.append(row)
        rows = rows[1:] # faccio slicings
        bboxes = []
        for row in rows:
            row = row.split(' ')
            target,x,y,w,h = row
            if int(target) == 1:
                box = [x,y,w,h]
                bboxes.append(box)
        # print(f'Ho trovato {len(bboxes)} boundary boxes')
        return bboxes
    return None

In [49]:
import os
directory = 'Annotations'
annotations = []
for root,dirs,files in os.walk(directory):
    for file in files:
        annotations.append(str(file))

In [50]:
# creo i set
train, test, val = assign_sets()

bboxes_train = []
for i in range(len(train)):
    bboxes = extract_boundary_box(train[i], annotations)
    if bboxes:
        # aggiungo alle Boundary Box anche il nome dell'immagine dove andare a cercare
        bboxes_train.append((train[i], bboxes))

bboxes_test = []
for i in range(len(test)):
    bboxes = extract_boundary_box(test[i], annotations)
    if bboxes:
        # aggiungo alle Boundary Box anche il nome dell'immagine dove andare a cercare
        bboxes_test.append([test[i], bboxes])

bboxes_val = []
for i in range(len(val)):
    bboxes = extract_boundary_box(val[i], annotations)
    if bboxes:
        # aggiungo alle Boundary Box anche il nome dell'immagine dove andare a cercare
        bboxes_val.append([val[i], bboxes])


print(f'Ci sono in totale {len(bboxes_train)} file di annotazioni per il train')
print(f'Ci sono in totale {len(bboxes_test)} file di annotazioni per il test')
print(f'Ci sono in totale {len(bboxes_val)} file di annotazioni per il validation')

Train set: 7000
Test set: 1000
Val set: 1000


Ci sono in totale 6919 file di annotazioni per il train
Ci sono in totale 983 file di annotazioni per il test
Ci sono in totale 991 file di annotazioni per il validation


In [51]:
example_image = bboxes_train[0]

In [52]:
path_image = example_image[0]
path_image = 'Images_Positive/' + path_image + '.jpg'
path_image

'Images_Positive/000040.jpg'

In [53]:
image = cv2.imread(path_image)
cv2.imshow('Immagine', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [54]:
bboxes = example_image[1]

In [55]:
len(bboxes)

30

Dipingo le boxes

In [56]:
bboxes

[['45', '235', '79', '318'],
 ['60', '209', '120', '356'],
 ['119', '214', '168', '336'],
 ['94', '220', '136', '326'],
 ['213', '201', '287', '381'],
 ['268', '201', '339', '378'],
 ['312', '192', '389', '380'],
 ['374', '226', '391', '267'],
 ['417', '225', '438', '279'],
 ['428', '222', '446', '266'],
 ['443', '224', '456', '258'],
 ['449', '227', '467', '271'],
 ['396', '217', '406', '241'],
 ['402', '210', '413', '238'],
 ['409', '217', '419', '241'],
 ['531', '222', '550', '287'],
 ['527', '217', '538', '245'],
 ['511', '213', '522', '240'],
 ['515', '211', '525', '237'],
 ['491', '216', '502', '243'],
 ['484', '209', '494', '235'],
 ['473', '211', '483', '235'],
 ['477', '211', '487', '236'],
 ['468', '212', '476', '235'],
 ['461', '210', '471', '236'],
 ['456', '212', '466', '240'],
 ['448', '209', '459', '237'],
 ['439', '210', '450', '237'],
 ['431', '211', '442', '241'],
 ['420', '213', '432', '242']]

In [57]:
new_image = image.copy()

In [58]:
for bbox in bboxes:
    for i in range(len(bbox)):
        bbox[i] = int(bbox[i])
    x,y,w,h = bbox
    cv2.rectangle(new_image, (x,y), (w,h), (0,0,255),2)

In [59]:
cv2.imshow('Image', new_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [60]:
# Specify the parameters for our HOG descriptor
win_size = (64, 128)
cell_size = (8, 8)
block_size = (16, 16)
block_stride = (8, 8)
num_bins = 9

# Set the parameters of the HOG descriptor using the variables defined above
hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, num_bins)

In [61]:
def subimage_processing(img):
    # faccio conversione in scala di grigi
    # cv2.imshow('Immagine originale', img)
    # cv2.waitKey(0)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # quindi applico un filtro gaussiano er ridurre eventuale rumore
    # poi applico una equilizzazione
    img = cv2.equalizeHist(img)
    # cv2.imshow('Immagine equilizzata', img)
    # cv2.waitKey(0)
    img = cv2.GaussianBlur(img,(3,3),0) # kernel size = 5x5, sigma_x = 0
    # Applica il filtro mediano per ridurre il rumore mantenendo i dettagli
    # img = cv2.medianBlur(img, 3)  # Usa una dimensione di kernel dispari
    # cv2.imshow('Immagine con filtro', img)
    # cv2.waitKey(0)
    # infine faccio reshape
    img = cv2.resize(img, (64, 128))  # Larghezza x Altezza
    # cv2.imshow('Immagine con reshape', img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return img

In [62]:
descriptors = []

In [63]:
sub_images = []

In [64]:
for bbox in bboxes:
    for i in range(len(bbox)):
        bbox[i] = int(bbox[i])
    x,y,w,h = bbox
    # cv2.rectangle(new_image, (x,y), (w,h), (0,0,255),2)
    sub_image = image[y:h, x:w].copy()
    sub_images.append(sub_image)
    cv2.imshow('Sottoimmagine prima del processing', sub_image)
    sub_image_proc = subimage_processing(sub_image)
    cv2.imshow('Sottoimmagine dopo il processing', sub_image_proc)
    cv2.waitKey(0)
    h = hog.compute(sub_image_proc)
    descriptors.append(h)
    # print(h)
cv2.destroyAllWindows()

In [65]:
import numpy as np

descriptors1 = np.array(descriptors)

x_new =  (x_orig – Media) / DeviazioneStandard

In [66]:
descriptors1.shape

(30, 3780)

In [67]:
descriptors1_0 = (descriptors1[0] - descriptors1[0].mean()) / descriptors1[0].std()

In [68]:
descriptors1_0.shape

(3780,)

In [69]:
descriptors1_1 = (descriptors1[1] - descriptors1[1].mean()) / descriptors1[1].std()

In [70]:
cv2.imshow('Immagine', sub_images[0])
cv2.imshow('Immagine2', sub_images[1])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [71]:
# prendo un campione negativo

In [72]:
directory_n = 'Images_negative'

images_negative = []

train_n = directory_n + '/train_neg'
test_n = directory_n + '/test_neg'
val_n = directory_n + '/val_neg'

neg_train_img = []
neg_test_img = []
neg_val_img = []


In [73]:
for root,dirs,files in os.walk(train_n):
    for file in files:
        images_negative.append(str(file))
        neg_train_img.append(str(file))

In [74]:
path = 'Images_negative/train_neg/'

In [75]:
neg_train_img[0]

'no_person__no_bike_184.png'

In [76]:
neg_example_img = cv2.imread(path + neg_train_img[0])
cv2.imshow('Immagine esempio', neg_example_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [77]:
import random

h,w,_ = neg_example_img.shape
portion_size = (150, 250) # da intedersi come (x,y)

tries = 10

sub_images = []
for i in range(tries):
    x = random.randint(0, w-1)
    y = random.randint(0, h-1)
    sub_neg_img = neg_example_img[y:y+portion_size[1], x:x+portion_size[0]].copy()
    sub_images.append(sub_neg_img)

In [78]:
descriptors2 = []

In [79]:
for sub_image in sub_images:
    cv2.imshow('Immagine esempio', sub_image)
    img_proc = subimage_processing(sub_image)
    cv2.imshow('Immagine pronta per applicare il descrittore', img_proc)
    h = hog.compute(img_proc)
    descriptors2.append(h)
    cv2.waitKey(0)
cv2.destroyAllWindows()

In [80]:
descriptors2 = np.array(descriptors2)
descriptors2_0 = (descriptors2[0] - descriptors2[0].mean()) / descriptors2[0].std()

In [81]:
descriptors2_0

array([-0.43344665, -0.68303657,  0.22132371, ...,  0.9163363 ,
        0.9163363 ,  0.3258165 ], dtype=float32)

In [82]:
descriptors1_0

array([ 0.75927794,  0.86132336, -0.02790849, ..., -1.0145924 ,
       -0.70543176,  2.080611  ], dtype=float32)

In [83]:
descriptors2.shape

(10, 3780)

In [84]:
descriptors1.shape

(30, 3780)

In [85]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(descriptors1[0], descriptors2[0])
mse

0.01977269

In [86]:
mse1 = mean_squared_error(descriptors1[0], descriptors1[1])
mse1

0.020094542

In [87]:
mse2 = mean_squared_error(descriptors1_0, descriptors2_0)
mse2

2.0178235

In [88]:
mse_same = mean_squared_error(descriptors1_0, descriptors1_1)
mse_same

1.7810416

In [89]:
mse3 =mean_squared_error(descriptors2_0, descriptors1_1)
mse3

2.1420677

In [90]:
descriptors1_3 = (descriptors1[2] - descriptors1[2].mean()) / descriptors1[2].std()

In [92]:
descriptors2_1 = (descriptors2[1] - descriptors2[1].mean()) / descriptors2[1].std()

In [96]:
mean_squared_error(descriptors1_1, descriptors1_3)

1.6267543